In [71]:
from torchdiffeq import odeint
from src.models.unet import Unet
from src.models.simple_model import SimpleModel
from torch.distributions.normal import Normal
from torchvision.utils import save_image

import torch
import torch.nn as nn

In [72]:
# model = Unet(channels=1, dim_mults=(1, 2, 4), dim=28)
model = SimpleModel()

In [99]:
checkpoint = torch.load("results/5-checkpoint.pth")

In [100]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [101]:
model.eval()

SimpleModel(
  (time_mlp): Sequential(
    (0): SinusoidalPositionEmbeddings()
    (1): Linear(in_features=28, out_features=1, bias=True)
    (2): GELU(approximate='none')
  )
  (net): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): GroupNorm(1, 4, eps=1e-05, affine=True)
    (2): SELU()
    (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): GroupNorm(1, 8, eps=1e-05, affine=True)
    (5): SELU()
    (6): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
)

In [102]:
base_distribution = Normal(0, 1)
x_0 = base_distribution.sample(sample_shape=(16, 1, 28, 28))

In [103]:
num_steps = 500
ts = torch.linspace(0., 1., num_steps)

In [104]:
@torch.no_grad()
def sample_euler(x_0: torch.Tensor, model: nn.Module, num_steps: int = 100):
    ts = torch.linspace(0., 1., num_steps)
    batch_size = x_0.shape[0]

    delta_t = ts[1] - ts[0]
    
    x_t = x_0
    for t in ts[1:]:
        x_t = x_t + model(x_t, t.repeat(batch_size)) * delta_t

    return x_t

In [105]:
output = sample_euler(x_0, model, num_steps)

In [106]:
output = (output + 1) * 0.5

In [107]:
output.shape

torch.Size([16, 1, 28, 28])

In [108]:
save_image(output, "samples.png", nrow=4)

In [84]:
# output